In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np
import matplotlib.patches as patches
import matplotlib
import tifffile
import os
import numpy as np
import sys
import dxchange
import matplotlib
import matplotlib.pyplot as plt
import scipy as sp
import scipy.ndimage as ndimage
%matplotlib inline

In [ ]:
import numpy as np
import tifffile
from holotomocupy.utils import *



In [ ]:
ss = 1024
nx, ny, nz = 3216, 3216, 2048
dtype = np.dtype("<f4")

bytes_per_voxel = dtype.itemsize
slice_bytes = ny * nx *nz* bytes_per_voxel

path = '/data2/vnikitin/brain/20251115/Y350a_HT_20nm_8dist_rec1234_/Y350a_HT_20nm_8dist_rec_cm_even_.vol'
with open(path, "rb") as f:
    buf = f.read(slice_bytes)
mrec0 = np.frombuffer(buf, dtype=dtype).reshape(nz,ny, nx)
mrec0=mrec0[nz//2-ss:nz//2+ss,ny//2-ss:ny//2+ss,nx//2-ss:nx//2+ss]

path = '/data2/vnikitin/brain/20251115/Y350a_HT_20nm_8dist_rec1234_/Y350a_HT_20nm_8dist_rec_cm_odd_.vol'
with open(path, "rb") as f:
    buf = f.read(slice_bytes)
mrec1 = np.frombuffer(buf, dtype=dtype).reshape(nz,ny, nx)
mrec1=mrec1[nz//2-ss:nz//2+ss,ny//2-ss:ny//2+ss,nx//2-ss:nx//2+ss]

In [ ]:
def halfbit(data, center):
    z, y, x = np.indices((data.shape))
    r = np.sqrt((x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2)
    r=r.astype(np.int32)
    nr = np.bincount(r.ravel())
    return (0.2071+1.9102/np.sqrt(nr))/(1.2071+0.9102/np.sqrt(nr)) 

def radial_profile(data, center):
    z, y, x = np.indices((data.shape))
    r = np.sqrt((x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2)
    r=r.astype(np.int32)
    

    tbinre = np.bincount(r.ravel(), data.real.ravel())
    tbinim = np.bincount(r.ravel(), data.imag.ravel())
    
    nr = np.bincount(r.ravel())
    radialprofile = (tbinre+1j*tbinim) / np.sqrt(nr)
    
    return radialprofile
from scipy.signal import savgol_filter

def smooth_savgol(data, window_size, poly_order):
    if window_size % 2 == 0 or window_size < 1:
      raise ValueError("Window size must be an odd positive integer")
    if poly_order >= window_size:
       raise ValueError("poly_order must be less than window_size")
    data = np.pad(data,(32,32),'edge')
    data = savgol_filter(data, window_size, poly_order)
    data=data[32:-32]
    return data

In [ ]:

wsize = ss

ff1 = np.zeros([2*wsize,2*wsize,2*wsize],dtype='complex64')
hbit = halfbit(ff1,np.array([2*wsize,2*wsize,2*wsize])//2)
np.save('hbit.npy',hbit)
len(hbit)


In [ ]:
mmrec0 = mrec0[mrec0.shape[0]//2-wsize:mrec0.shape[0]//2+wsize,mrec0.shape[1]//2-wsize:mrec0.shape[1]//2+wsize,mrec0.shape[2]//2-wsize:mrec0.shape[2]//2+wsize]
mmrec1 = mrec1[mrec1.shape[0]//2-wsize:mrec1.shape[0]//2+wsize,mrec1.shape[1]//2-wsize:mrec1.shape[1]//2+wsize,mrec1.shape[2]//2-wsize:mrec1.shape[2]//2+wsize]
mmrec0 = np.fft.fftshift(np.fft.fftn(mmrec0))
mmrec1 = np.fft.fftshift(np.fft.fftn(mmrec1))

frc2 = radial_profile(mmrec0*np.conj(mmrec1),np.array(mmrec0.shape)//2)/\
    np.sqrt(radial_profile(np.abs(mmrec0)**2,np.array(mmrec0.shape)//2)*radial_profile(np.abs(mmrec1)**2,np.array(mmrec0.shape)//2))

# a = np.hamming(2*len(frc2))
# a = a[len(a)//2:]
# frc2*=a
# frc2=np.abs(frc2)[:wsize]
np.save(f'pfrc2.npy',frc2)



In [ ]:
frc2.shape

In [ ]:
import matplotlib as mpl

mpl.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],  # fallbacks
})

mpl.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
    "mathtext.fontset": "stix",   # Times-like math
    # optional:
    # "mathtext.default": "regular",
})
mpl.rcParams['font.size'] = 24  # optional
mpl.rcParams["xtick.labelsize"] = 22
mpl.rcParams["ytick.labelsize"] = 22

In [ ]:

fig, ax = plt.subplots(1,1,figsize=(10,5))


frc2c=smooth_savgol(frc2, 11, 5)
hbit=np.load('hbit.npy')

ax.plot(frc2c[:].real,linewidth=1.5, label=f'proposed')

ax.plot(hbit[:wsize],linewidth=1.5,label='1/2-bit')

ax.grid()
ax.set_ylim([0,1])
lg = plt.legend(loc="upper right")
lg.get_title().set_fontsize(16)
plt.ylabel('FSC',rotation=90)
ax2 = ax.secondary_xaxis('top')
ax.set_xlabel(f'Spatial frequency (1/$\mu$m)')
ax2.set_xlabel('Spatial resolution (nm)')
ax.set_xticks(np.linspace(0,wsize,6),np.int32(np.linspace(0,1/(2*20e-9),6)/1e6))
ax2.set_xticks(np.linspace(0,wsize,6),(np.int32(20000*2**bin/np.linspace(1,10,6))/100))
# # plt.show()
# plt.savefig(f'{iter}_{wsize}.png',dpi=300)

In [ ]:
128e-6/40e-9